# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [49]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys
from collections import deque  

## 2. INICIALIZAR MEDIAPIPE

In [50]:
# Configuración inicial global
mp_hands = mp.solutions.hands

# Optimizar MediaPipe
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.45,  # Reducir confianza
    min_tracking_confidence=0.45,
    model_complexity=0  # Menor complejidad
)

mp_draw = mp.solutions.drawing_utils
dataset_dir = "dataset_11_90"
model_path = "gesture_model_me_10_90.h5"
sequence_length = 90
total_landmarks = 126
gestures = []
X_mean = None
X_std = None

num_camara = 0

## 3. FUNCIONES PRINCIPALES

In [51]:
# Funciones principales
def init_system():
    global gestures
    os.makedirs(dataset_dir, exist_ok=True)
    gestures = get_existing_gestures()
    
def get_existing_gestures():
    return [d for d in os.listdir(dataset_dir) 
           if os.path.isdir(os.path.join(dataset_dir, d))]

## 4. DETECCION DE MANO

In [52]:
def detect_hands():
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Detección de Manos", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

## 5. RECOLLECION DE DATOS 

In [53]:
def collect_data():
    global gestures
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))
    
    save_dir = os.path.join(dataset_dir, gesture)
    os.makedirs(save_dir, exist_ok=True)

    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")
    print("Mantenga la seña frente a la cámara...")
    
    cap = cv2.VideoCapture(0)
    sequence = []
    counter = 0

    #NUEVO Configurar ventana de landmarks
    landmark_window_name = "Landmarks en Tiempo Real"
    cv2.namedWindow(landmark_window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(landmark_window_name, 640, 480)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        #NUEVO Crear canvas para landmarks
        landmark_canvas = np.zeros((480, 640, 3), dtype=np.uint8)  # Canvas negro 640x480

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            all_landmarks = []

            #NUEVO Dibujar landmarks en el canvas
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar en el canvas negro
                mp_draw.draw_landmarks(
                    landmark_canvas,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2)
                )
            
            # Extraer coordenadas para el dataset
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])
            
            # Rellenar si solo hay una mano
            if len(results.multi_hand_landmarks) < 2:
                all_landmarks += [0.0] * 63
            
            sequence.append(all_landmarks)

            # Dibujar en el frame original
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        if len(sequence) == sequence_length:
            np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)
            counter += 1
            sequence = []
            print(f"Secuencias capturadas: {counter}/{num_sequences}")

        #NUEVO Mostrar información en ambas ventanas
        info_text = f"Secuencias: {counter}/{num_sequences}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(landmark_canvas, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        cv2.imshow("Recolección de Datos", frame)
        #NEUVO
        cv2.imshow(landmark_window_name, landmark_canvas)
        if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:
            break

    cap.release()
    cv2.destroyAllWindows()
    gestures = get_existing_gestures()
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")

## 6. CARGA DE DATOS

In [54]:
def load_data():
    X = []
    y = []
    
    for label_idx, gesture in enumerate(gestures):
        gesture_dir = os.path.join(dataset_dir, gesture)
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            sequence = np.load(seq_path)
            
            if sequence.shape == (sequence_length, total_landmarks):
                X.append(sequence)
                y.append(label_idx)
    
    return np.array(X), np.array(y), gestures


## 7. ENTRENAMIENTO DEL MODELO

In [55]:
def train_model():
    global X_mean, X_std, gestures
    
    # 1. Verificar datos de entrenamiento
    gestures = get_existing_gestures()
    if not gestures:
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return

    # 2. Cargar y preparar datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data()
    y = tf.keras.utils.to_categorical(y)

    # 3. Calcular parámetros de normalización
    X_mean = np.mean(X, axis=(0, 1))
    X_std = np.std(X, axis=(0, 1))
    X = (X - X_mean) / X_std  # Normalización

    # 4. Guardar parámetros de normalización
    np.savez('normalization_params_90.npz', mean=X_mean, std=X_std)
    
    # 5. Arquitectura optimizada del modelo
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # 6. Compilación y entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        #callbacks=[tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)],
        verbose=1
    )

    # 7. Guardar modelo y resultados
    model.save(model_path)
    print(f"\nModelo guardado en {model_path}")
    
    # 8. Conversión a TFLite con configuraciones especiales
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False
    
    try:
        tflite_model = converter.convert()
        with open('model_quantized_90.tflite', 'wb') as f:
            f.write(tflite_model)
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")
    
    # Mostrar métricas finales
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [56]:
def convert_to_tflite():
    try:
        # Cargar el modelo entrenado
        model = tf.keras.models.load_model(model_path)
        
        # Configurar el conversor con parámetros especiales
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Añadir estas 3 líneas clave para compatibilidad con LSTM
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter._experimental_lower_tensor_list_ops = False
        converter.allow_custom_ops = True  # Permitir operaciones personalizadas
        
        # Realizar la conversión
        tflite_model = converter.convert()
        
        # Guardar el modelo cuantizado
        with open('model_quantized_90.tflite', 'wb') as f:
            f.write(tflite_model)
            
        print("\n✅ Conversión a TFLite exitosa!")
        
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")
        print("Posibles soluciones:")
        print("1. Verifique que el modelo .h5 existe")
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")
        print("3. Reinicie el runtime/kernel")

def representative_dataset_gen():
    # Generador de datos de ejemplo para calibración
    for _ in range(100):
        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]

## 8. EVALUACION DEL MODELO

In [ ]:
from threading import Thread
def evaluate():
    if not os.path.exists("model_quantized_90.tflite"):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    
    # 1. Cargar parámetros y modelo
    try:
        with np.load('normalization_params_90.npz') as data:
            X_mean = data['mean']
            X_std = data['std']
            
        interpreter = tf.lite.Interpreter(model_path="model_quantized_90.tflite")
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]  # Mantener [0]
        output_details = interpreter.get_output_details()[0]
    except Exception as e:
        print(f"\nError crítico: {str(e)}")
        return

    # 2. Configuración de cámara
    cap = cv2.VideoCapture(0)  # Forzar backend DirectShow
    #import time
    #time.sleep(2)  # Permitir inicialización de cámara
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    #cap.set(cv2.CAP_PROP_FPS, 30)
    
    if not cap.isOpened():
        print("\n¡No se puede acceder a la cámara!")
        return

    # 3. Variables de estado
    current_gesture = None  # Variable para almacenar la última seña detectada
    current_confidence = 0.0
    sequence = deque(maxlen=sequence_length)  # Usar deque
    #last_gesture = None
    frame_counter = 0
    #prediction_active = False

    # 4. Bucle principal
    while True:
        # 4.1 Capturar frame
        ret, frame = cap.read()
        if not ret:
            print("\nError en captura de frame")
            break
            
        frame_counter += 1
        
        # 4.2 Procesamiento cada 2 frames
        if frame_counter % 2 != 0:
            continue
            
        # 4.3 Detección de landmarks
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            # 4.4 Extraer landmarks
            landmarks = []
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            
            # Rellenar si solo hay una mano
            if len(results.multi_hand_landmarks) < 2:
                landmarks += [0.0] * (total_landmarks - len(landmarks))
                
            # El deque ya maneja automáticamente el tamaño máximo gracias a maxlen
            sequence.append(landmarks)  # Esto mantendrá solo los últimos 'sequence_length' elementos
            
            
            # 4.5 Dibujar landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                

                # Añadir verificación de dimensión de entrada
            if input_details['shape'][1] != sequence_length:
                print("\nError: Dimensiones del modelo no coinciden con la configuración")
                return
            
            # 4.6 Realizar predicción cada 1 segundo (30 frames)
            if len(sequence) == sequence_length and frame_counter % 15 == 0:
                try:
                    # Preprocesamiento CORREGIDO
                    seq_array = np.array(sequence)
                    seq_array = (seq_array - X_mean) / (X_std + 1e-7)  # +epsilon para seguridad
                    
                    # Asegurar 3 dimensiones: [batch_size=1, sequence_length, features]
                    input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                    
                    # Debug: Verificar forma
                    
                    print("Forma de entrada:", input_data.shape)  # Debe mostrar (1, 90, 126)

                    # Inferencia
                    interpreter.set_tensor(input_details['index'], input_data)
                    interpreter.invoke()
                    prediction = interpreter.get_tensor(output_details['index'])[0]
                    
                    # Procesar resultados
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[predicted_idx]
                    
                    if confidence > 0.65:
                        current_gesture = gestures[predicted_idx]  # Actualizar solo si hay alta confianza
                        current_confidence = confidence
                        print("Nueva seña detectada:", current_gesture)
                        
                        
                        cv2.putText(frame, f"{current_gesture} ({confidence:.2%})", 
                            (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2) #el 0,255,0 es verde
                        
                        print("La seña es: ", current_gesture)

                            
                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None  # Resetear si hay error
                    
        if current_gesture:
            cv2.putText(frame, 
              f"{current_gesture} ({current_confidence:.2%})", 
              (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Mostrar frame reducido
        cv2.imshow("Predicciones", cv2.resize(frame, (640, 480)))
        
        
        # 4.8 Salir con ESC
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 5. Limpieza
    cap.release()
    cv2.destroyAllWindows()
    print("\nSistema de evaluación detenido")

## 9. REENTRENAR GESTO

In [58]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## 10. MENU

In [59]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Evaluar")
        print("5. Reentrenar Gesto")
        print("6. Convertir a TFLite")  # Nueva opción
        print("7. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':
            retrain_gesture()
        elif choice == '6':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '7':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [60]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. Convertir a TFLite
7. Salir

Cargando datos y preparando el entrenamiento...

Iniciando entrenamiento...
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 174ms/step - accuracy: 0.0975 - loss: 2.0510 - val_accuracy: 0.0476 - val_loss: 2.2832
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.3287 - loss: 1.8515 - val_accuracy: 0.0952 - val_loss: 2.2046
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5288 - loss: 1.6891 - val_accuracy: 0.1667 - val_loss: 2.1475
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7507 - loss: 1.5224 - val_accuracy: 0.1905 - val_loss: 2.1504
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7682 - loss: 1.4101 - val_accuracy: 0.2381 - val_loss: 2.2102
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8217 - loss: 1.2840 - va


Modelo guardado en gesture_model_me_10_90.h5
INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmppw_y5mz7\assets


INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmppw_y5mz7\assets


Saved artifact at 'C:\Users\juanp\AppData\Local\Temp\tmppw_y5mz7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 90, 126), dtype=tf.float32, name='keras_tensor_33')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2818460362608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349900624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349902032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349547584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349559376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349557792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818349558144: TensorSpec(shape=(), dtype=tf.resource, name=None)

Modelo TFLite exportado exitosamente
Precisión de validación final: 28.57%

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convert

INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpv7wiawfi\assets


INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpv7wiawfi\assets


Saved artifact at 'C:\Users\juanp\AppData\Local\Temp\tmpv7wiawfi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 90, 126), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2818494001952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493957920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493966720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493965664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493969184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493968128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2818493967424: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ Conversión a TFLite exitosa!

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. 